In [1]:
%pylab inline
import os
import gc
import glob
import numpy as np
import astropy.io.fits as pyfits
import maputil as mapUtil
from configparser import ConfigParser
from pixel3D import cartesianGrid3D

Populating the interactive namespace from numpy and matplotlib


# Estimate average poz and lensing kernel

In [103]:
configName  =   'wrkDir/config-nl15.ini'
pDict={'dempz': 'demp',\
       'dnnz': 'dnn',\
       'mizuki': 'mizuki'}

In [104]:
nz=10
key='dempz'
colname=pDict[key]
pzDir=os.path.join('datDir/catalog_tracts/','XMM_tracts/')
pbins= pyfits.getdata(os.path.join(pzDir,'8522_pz_pdf_%s.fits' %key),ext=2)['BINS']
nbin=len(pbins)
for fieldname in mapUtil.field_names:
    inDir  =   'wrkDir/reGaus_catalog/%s_groups/' %fieldname
    outDir =   'wrkDir/reGaus_pixel3D/%s_groups/' %fieldname
    if not os.path.isdir(outDir):
        os.mkdir(outDir)
    fnames=glob.glob(os.path.join(inDir,'src-group-*.fits'))
    for fname in fnames:
        gn=fname.split('-')[-1].split('.')[0]
        glab=fieldname+"-%s"%gn
        # poz average
        outname1=os.path.join(outDir,'group-%s-%s-10bins.fits' %(gn,key))
        if os.path.isfile(outname1):
            poz_ave=pyfits.getdata(outname1,ext=0)
            bounds=pyfits.getdata(outname1,ext=1)['zbounds']
        else:
            data=pyfits.getdata(fname)
            mask=(~np.isnan(data['%s_zbest' %colname]))\
                &(data['%s_zbest_std'%colname]<1.0)\
                &(data['%s_zbest'%colname]<3.0)\
                &(data['%s_zbest'%colname]>0.05)
            poz_best=data['%s_zbest'%colname]
            poz_best=poz_best[mask]
            bounds=np.percentile(poz_best,np.linspace(0,100,nz+1))
            pfname=os.path.join(inDir,'poz-%s-group-%s.fits' %(key,gn))
            pata=pyfits.getdata(pfname)
            pata=pata[mask]
            poz_ave=np.zeros((nz,nbin))
            for iz in range(nz):
                tmp_msk=(poz_best>=bounds[iz])&(poz_best<bounds[iz+1])
                poz_ave[iz]=np.average(pata['PDF'][tmp_msk],axis=0)
            out=pyfits.HDUList([pyfits.PrimaryHDU(poz_ave),\
            pyfits.BinTableHDU.from_columns([pyfits.Column(name='zbounds',array=bounds,format='D')])])
            out.writeto(outname1,overwrite=True)
            del data,pata,out
            gc.collect()
        outname2=os.path.join(outDir,'group-%s-%s-10bins-pozave.png' %(gn,key))
        mapUtil.plotPozAve(glab,pbins,poz_ave,outname2)
        
        # configuration
        parser      =   ConfigParser()
        parser.read(configName)
        parser.set('sourceZ','zbound','[%s]' %(','.join([str(bb) for bb in bounds])))
        gridInfo    =   cartesianGrid3D(parser)
        # lensing kernel
        outname3=os.path.join(outDir,'group-%s-%s-10bins-lensker.fits' %(gn,key))
        if os.path.isfile(outname3):
            lensKer2=pyfits.getdata(outname3,ext=0)
            lensKer1=pyfits.getdata(outname3,ext=1)
        else:
            lensKer1= gridInfo.lensing_kernel()
            lensKer2= gridInfo.lensing_kernel(poz_grids=pbins,poz_ave=poz_ave)
            out=pyfits.HDUList([pyfits.PrimaryHDU(lensKer2),pyfits.ImageHDU(lensKer1)])
            out.writeto(outname3,overwrite=True)
        outname4=os.path.join(outDir,'group-%s-%s-10bins-lensker.png' %(gn,key))
        mapUtil.plotLensKernel(glab,gridInfo.zcgrid,lensKer1,lensKer2,outname4)